In [1]:
# Importación de librerías
import numpy as np
import tensorflow as tf
import cv2
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Cargar el dataset CIFAR-10
(imagenes_entrenamiento, etiquetas_entrenamiento), (imagenes_prueba, etiquetas_prueba) = cifar10.load_data()
print("Datos de entrenamiento: ", len(imagenes_entrenamiento))
print("Datos de validación: ", len(imagenes_prueba))

# Función para la segmentación de imágenes utilizando umbralización
def segmentar_imagen(imagen):
    # Convertir la imagen a escala de grises
    gris = cv2.cvtColor(imagen, cv2.COLOR_RGB2GRAY)
    # Aplicar umbralización
    _, imagen_segmentada = cv2.threshold(gris, 128, 255, cv2.THRESH_BINARY)
    # Convertir la imagen segmentada a tres canales
    imagen_segmentada = cv2.cvtColor(imagen_segmentada, cv2.COLOR_GRAY2RGB)
    return imagen_segmentada

# Aplicar la segmentación a las imágenes de entrenamiento y prueba
imagenes_entrenamiento_segmentadas = np.array([segmentar_imagen(img) for img in imagenes_entrenamiento])
imagenes_prueba_segmentadas = np.array([segmentar_imagen(img) for img in imagenes_prueba])

# Procesamiento de datos
print("\nValor original de pixel: ", imagenes_entrenamiento_segmentadas[0][0][0][0])
media = np.mean(imagenes_entrenamiento_segmentadas, axis=(0, 1, 2, 3))
desviacion_estandar = np.std(imagenes_entrenamiento_segmentadas, axis=(0, 1, 2, 3))
imagenes_entrenamiento_segmentadas = (imagenes_entrenamiento_segmentadas - media) / (desviacion_estandar + 1e-7)
imagenes_prueba_segmentadas = (imagenes_prueba_segmentadas - media) / (desviacion_estandar + 1e-7)
print("Valor procesado de pixel: ", imagenes_entrenamiento_segmentadas[0][0][0][0])

# Aumento de datos
aumento_datos = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)
aumento_datos.fit(imagenes_entrenamiento_segmentadas)

# Convertir etiquetas a formato categórico
clases = 10

print("\nEtiqueta original: ", etiquetas_entrenamiento[0][0])
etiquetas_entrenamiento = tf.keras.utils.to_categorical(etiquetas_entrenamiento, clases)
etiquetas_prueba = tf.keras.utils.to_categorical(etiquetas_prueba, clases)
print("Etiqueta categórica: ", etiquetas_entrenamiento[0])

# Construcción del modelo
modelo = Sequential([
    # Feature Extraction
    Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4), input_shape=imagenes_entrenamiento_segmentadas.shape[1:]),
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.4),

    Flatten(),
    # Classification
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(clases, activation='softmax')
])

# Compilación del modelo
modelo.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Configuración de callbacks (Post-processing)
detencion_temprana = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduccion_tasa_aprendizaje = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)

# Entrenar el modelo
tamano_lote = 64
historial = modelo.fit(aumento_datos.flow(imagenes_entrenamiento_segmentadas, etiquetas_entrenamiento, batch_size=tamano_lote),
                    steps_per_epoch=imagenes_entrenamiento_segmentadas.shape[0] // tamano_lote,
                    epochs=100,
                    validation_data=(imagenes_prueba_segmentadas, etiquetas_prueba),
                    callbacks=[detencion_temprana, reduccion_tasa_aprendizaje])

# Evaluación del modelo
perdida, precision = modelo.evaluate(imagenes_prueba_segmentadas, etiquetas_prueba)
print(f"Pérdida: {perdida}")
print(f"Precisión: {precision}")




Datos de entrenamiento:  50000
Datos de validación:  10000

Valor original de pixel:  0
Valor procesado de pixel:  -0.8991011972023358

Etiqueta original:  6
Etiqueta categórica:  [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
Epoch 1/100
781/781 [==============================] - 24s 26ms/step - loss: 2.6052 - accuracy: 0.1233 - val_loss: 2.2360 - val_accuracy: 0.2048 - lr: 1.0000e-04
Epoch 2/100
781/781 [==============================] - 19s 25ms/step - loss: 2.2806 - accuracy: 0.1598 - val_loss: 2.1251 - val_accuracy: 0.2640 - lr: 1.0000e-04
Epoch 3/100
781/781 [==============================] - 24s 30ms/step - loss: 2.1912 - accuracy: 0.1982 - val_loss: 1.9932 - val_accuracy: 0.3020 - lr: 1.0000e-04
Epoch 4/100
781/781 [==============================] - 41s 52ms/step - loss: 2.1221 - accuracy: 0.2291 - val_loss: 1.9082 - val_accuracy: 0.3249 - lr: 1.0000e-04
Epoch 5/100
781/781 [==============================] - 19s 24ms/step - loss: 2.0608 - accuracy: 0.2572 - val_loss: 1.9158 - val_accuracy: 0.

KeyboardInterrupt: 

In [ ]:
# Guardar el modelo entrenado
modelo.save('cifar10_segmentado.h5')